In [1]:
%load_ext autoreload
%autoreload 2

import bnl
import frameless_eval as fle
import mir_eval
import random
import plotly.express as px
import pandas as pd

import matplotlib.pyplot as plt
from dataclasses import replace

def px_metrics_bar(metrics, title="mir_eval hierarchy metrics"):
    # metrics: dict or OrderedDict of name -> float
    df = pd.Series(metrics, dtype=float).rename_axis("metric").reset_index(name="score")
    fig = px.bar(df, x="metric", y="score", title=title, text="score", range_y=[0,1.2])
    fig.update_traces(texttemplate="%{text:.3f}", textposition="outside")
    fig.update_layout(xaxis_tickangle=-45, width=450, height=300, margin=dict(l=10, r=10, t=50, b=10))
    return fig

In [2]:
slm_ds = bnl.data.Dataset(manifest_path="~/data/salami/metadata.csv")

In [9]:
track = slm_ds[11]
ref = list(track.refs.values())[0]
est = track.ests['mu1gamma9'].align(ref)

ref.plot().show()
est.plot().show()

In [10]:
est_contour_depth = est.contour('depth')
est_contour_count = est.contour('count')
est_contour_weight = est.contour('prob')

In [50]:
est_by_depth = est_contour_depth.level('mean_shift', bw=0.15).to_ms(name='depth').prune_layers().scrub_labels()
est_by_count = est_contour_count.level('mean_shift', bw=0.15).to_ms(name='count').prune_layers().scrub_labels()
est_by_weight = est_contour_weight.level('mean_shift', bw=0.15).to_ms(name='weight').prune_layers().scrub_labels()

In [48]:
est_by_depth_tall = est_contour_depth.level().to_ms(name='depth').prune_layers().scrub_labels()
est_by_count_tall = est_contour_count.level().to_ms(name='count').prune_layers().scrub_labels()
est_by_weight_tall = est_contour_weight.level().to_ms(name='weight').prune_layers().scrub_labels()

In [51]:
est_by_depth.plot().update_layout(width=450).show()
est_by_count.plot().update_layout(width=450).show()
est_by_weight.plot().update_layout(width=450).show()

In [52]:
import warnings
warnings.filterwarnings('ignore')

print("adjusted levels")
for e, title in zip(
    [est, est_by_depth, est_by_count, est_by_weight], 
    ['raw', 'depth', 'count', 'weight']
):
    hier_score = mir_eval.hierarchy.evaluate(ref.itvls, ref.labels, e.itvls, e.labels)
    del hier_score['L-Precision']
    del hier_score['L-Recall']
    del hier_score['L-Measure']
    px_metrics_bar(hier_score, title=title).show()

adjusted levels


In [49]:
import warnings
warnings.filterwarnings('ignore')

print('levels unadjusted, tall hierarchies')
for e, title in zip(
    [est, est_by_depth_tall, est_by_count_tall, est_by_weight_tall], 
    ['raw', 'depth_tall', 'count_tall', 'weight_tall']
):
    hier_score = mir_eval.hierarchy.evaluate(ref.itvls, ref.labels, e.itvls, e.labels)
    del hier_score['L-Precision']
    del hier_score['L-Recall']
    del hier_score['L-Measure']
    px_metrics_bar(hier_score, title=title).show()

levels unadjusted, tall hierarchies


In [24]:
# Let's see if we can see the effect of cleaning up boundaries at all
est_contour_depth.plot().show()
est_contour_depth.clean('absorb', window=2).plot().show()
est_contour_depth.clean('kde', bw=1).plot().show()

In [25]:
est_contour_count.plot().show()
est_contour_count.clean('absorb', window=2).plot().show()
est_contour_count.clean('kde', bw=1).plot().show()

In [54]:
est_contour_weight.plot().show()
est_contour_weight.clean('absorb').plot().show()
est_contour_weight.clean('kde').plot().show()

In [33]:
est_contour_weight.level().plot().show()
est_contour_weight.level('mean_shift', bw=0.1).plot().show()

In [36]:
est_contour_count.level().plot().show()
est_contour_count.level('mean_shift', bw=0.1).plot().show()

In [38]:
est_contour_weight.level().plot().show()
est_contour_weight.level('mean_shift', bw=0.1).plot().show()

In [3]:
from pqdm.processes import pqdm
from bnl.exp import test_mono_casting_effects as tmce
import bnl

slm_ds = bnl.data.Dataset(manifest_path="~/data/salami/metadata.csv")
list_of_dfs = pqdm(slm_ds, tmce, n_jobs=8)

QUEUEING TASKS | :   0%|          | 0/1359 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1359 [00:00<?, ?it/s]

Running experiment for track 6...
Running experiment for track 7...
Running experiment for track 4...
Running experiment for track 10...
Running experiment for track 8...
Running experiment for track 3...
Running experiment for track 5...
Running experiment for track 2...
Results for track 6 saved to ./monocasting_results/6.feather
Loading existing results for track 11 from ./monocasting_results/11.feather
Running experiment for track 12...
Results for track 7 saved to ./monocasting_results/7.feather
Running experiment for track 13...
Results for track 8 saved to ./monocasting_results/8.feather
Running experiment for track 14...
Results for track 4 saved to ./monocasting_results/4.feather
Running experiment for track 15...
Results for track 2 saved to ./monocasting_results/2.feather
Running experiment for track 16...
Results for track 5 saved to ./monocasting_results/5.feather
Running experiment for track 18...
Results for track 13 saved to ./monocasting_results/13.feather
Running expe

/Users/tomxi/miniforge3/envs/py311/lib/python3.11/site-packages/mir_eval/hierarchy.py:460: UserWarning: Segment hierarchy is inconsistent at level 1
  warnings.warn(
/Users/tomxi/miniforge3/envs/py311/lib/python3.11/site-packages/mir_eval/hierarchy.py:460: UserWarning: Segment hierarchy is inconsistent at level 1
  warnings.warn(
/Users/tomxi/miniforge3/envs/py311/lib/python3.11/site-packages/mir_eval/hierarchy.py:460: UserWarning: Segment hierarchy is inconsistent at level 1
  warnings.warn(
/Users/tomxi/miniforge3/envs/py311/lib/python3.11/site-packages/mir_eval/hierarchy.py:460: UserWarning: Segment hierarchy is inconsistent at level 1
  warnings.warn(
/Users/tomxi/miniforge3/envs/py311/lib/python3.11/site-packages/mir_eval/hierarchy.py:460: UserWarning: Segment hierarchy is inconsistent at level 1
  warnings.warn(
/Users/tomxi/miniforge3/envs/py311/lib/python3.11/site-packages/mir_eval/hierarchy.py:460: UserWarning: Segment hierarchy is inconsistent at level 1
  warnings.warn(
/Use

Results for track 192 saved to ./monocasting_results/192.feather
Running experiment for track 198...


...

/Users/tomxi/miniforge3/envs/py311/lib/python3.11/site-packages/mir_eval/hierarchy.py:460: UserWarning: Segment hierarchy is inconsistent at level 1
  warnings.warn(


Results for track 1203 saved to ./monocasting_results/1203.feather
Running experiment for track 1221...
Results for track 1220 saved to ./monocasting_results/1220.feather
Running experiment for track 1222...
Results for track 1215 saved to ./monocasting_results/1215.feather
Running experiment for track 1223...


/Users/tomxi/miniforge3/envs/py311/lib/python3.11/site-packages/mir_eval/hierarchy.py:460: UserWarning: Segment hierarchy is inconsistent at level 1
  warnings.warn(


Results for track 1219 saved to ./monocasting_results/1219.feather
Running experiment for track 1224...
Results for track 1198 saved to ./monocasting_results/1198.feather
Running experiment for track 1226...
Results for track 1218 saved to ./monocasting_results/1218.feather
Running experiment for track 1228...
Results for track 1226 saved to ./monocasting_results/1226.feather
Running experiment for track 1229...
Results for track 1223 saved to ./monocasting_results/1223.feather
Running experiment for track 1230...
Results for track 1221 saved to ./monocasting_results/1221.feather
Running experiment for track 1231...
Results for track 1222 saved to ./monocasting_results/1222.feather
Running experiment for track 1232...
Results for track 1224 saved to ./monocasting_results/1224.feather
Running experiment for track 1234...
Results for track 1228 saved to ./monocasting_results/1228.feather
Running experiment for track 1235...
Results for track 1229 saved to ./monocasting_results/1229.feath

COLLECTING RESULTS | :   0%|          | 0/1359 [00:00<?, ?it/s]

In [12]:
type(list_of_dfs[1]) is pd.DataFrame

True

In [16]:
valid_dfs = [df for df in list_of_dfs if type(df) is pd.DataFrame]

In [19]:
len(valid_dfs)

1350

In [17]:
import pandas as pd
all_results_df = pd.concat(valid_dfs, ignore_index=True)

In [ ]:
all_results_df

In [77]:
est[-1].plot()